This is my second hands on experience with GEE Python API.
In this, I shall be importing a DEM dataset and Global Forest cover dataset to make a primary visual assessment of the Andean Region. 
In the coming days, i shall extend this to more sophisticated analysis.

Step 1: Let's import and initialize the EE Python API

In [3]:
import ee

ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=0tdjX8dep-aJuxFRUsmIsuGwhM1dGUMYd228AEQEE6U&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWiVRYDfEVR7rolqXIAS0ojszKg76YURu6Boxjf4h59akYbPhvJK6s4

Successfully saved authorization token.


In [4]:
#After succesful authentication, we can initialize the API to start working.
ee.Initialize()

In [73]:
geojson ={
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -83.671875,
              -54.05938788662357
            ],
            [
              -35.33203125,
              -54.87660665410867
            ],
            [
              -33.39843750000001,
              -8.233237111274553
            ],
            [
              -36.2109375,
              12.039320557540572
            ],
            [
              -83.671875,
              12.382928338487408
            ],
            [
              -83.671875,
              -54.05938788662357
            ]
          ]
        ]
      }
    }
  ]
}

In [74]:
coords = geojson['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

In [75]:
dem = ee.Image('USGS/SRTMGL1_003').clip(aoi)

In [76]:
dataset = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/FNF').first()
fnf = dataset.clip(aoi)

fnfVis = {
  'bands': ['fnf'],
  'min': 1.0,
  'max': 3.0,
  'palette': ['006400', 'FEFF99', '0000FF']}

#rgb_f = forestNonForest.visualize(**forestNonForestVis) 
#url_f = fnf.getThumbURL({'min': 1.0, 'max':3.0})
#print(url_f)

In [77]:
# Make pixels with elevation below sea level transparent.
elv_img = dem.updateMask(dem.gt(1800))

# Display the thumbnail of styled elevation in France.
elevis = {
    'min': 1800, 'max': 8000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

rgb = elv_img.visualize(**elevis) 
url = rgb.getThumbURL({'dimensions': 512})
print(url)    

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/06e34a23df4834d468deb5884ab57f4b-6f8c0bc091d2be3bb58da9261879db74:getPixels


In [78]:
import folium

# Define the center of our map.
lat, lon = -32.338200, -70.051575

my_map = folium.Map(location=[lat, lon], zoom_start=4)
my_map

In [79]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [80]:
my_map = folium.Map(location=[lat, lon], zoom_start=7)

In [81]:
exaggeration = 20
hillshade = ee.Terrain.hillshade(elv_img.multiply(exaggeration))
hillvis = {'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

In [82]:


my_map.add_ee_layer(elv_img, elevis, 'Elevation')
my_map.add_ee_layer(hillshade, hillvis, 'Hillshade')
# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

In [83]:
my_map.save('SA_SRTM_interactive.html')

In [84]:

my_map_2 = folium.Map(location=[lat, lon], zoom_start=7)

In [85]:
# Add the land cover to the map object.

my_map_2.add_ee_layer(fnf, fnfVis, 'Forest Cover')
# Add a layer control panel to the map.
my_map_2.add_child(folium.LayerControl())

# Display the map.
display(my_map_2)

In [86]:
# Load an image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# Define the visualization parameters.
image_viz_params = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0,
    'max': 0.5,
    'gamma': [0.95, 1.1, 1]
}

# Define a map centered on San Francisco Bay.
map_l8 = folium.Map(location=[37.5010, -122.1899], zoom_start=10)

# Add the image layer to the map and display it.
map_l8.add_ee_layer(image, image_viz_params, 'false color composite')
display(map_l8)